<a href="https://colab.research.google.com/github/44REAM/CEB-image_processing/blob/main/image1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import cv2

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
ct_image = cv2.imread('data/ct.jpeg')
print(f"image shape: {ct_image.shape}")

ct_image = ct_image[:,:,0]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (10,5))


axes[0].imshow(ct_image, cmap = 'gray')
sns.histplot(ct_image.ravel(), bins=20, ax = axes[1])

In [ ]:
# https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#gaa9e58d2860d4afa658ef70a9b1115576
ct_image_thres = cv2.threshold(ct_image, 20, 200, cv2.THRESH_BINARY)
plt.imshow(ct_image_thres[1], cmap = 'gray')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (10,5))

ct_equ = cv2.equalizeHist(ct_image)
axes[0].imshow(ct_image, cmap = 'gray')
axes[1].imshow(ct_equ, cmap = 'gray')


In [ ]:
def threshold(image, lower_bound = None, upper_bound = None, new_intensity = 0):
    assert lower_bound is not None or upper_bound is not None
    if lower_bound is not None:
        
    ct_image[(ct_image > lower_bound) & (ct_image < upper_bound)] = new_intensity

In [ ]:
ct_image[(ct_image > 20) & (ct_image < 200)] = 0